___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [45]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [123]:
import tweepy
import math
import os.path
import pandas as pd
import json
import nltk
import numpy as np


___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@YMartins123***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [102]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [103]:
#Produto escolhido:
produto = 'Samsung'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [104]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

NameError: name 'shuffle' is not defined

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [110]:
#Limpando as mensagens 

import string
import random
data=pd.read_excel("Samsung.xlsx")
datatest=pd.read_excel("Samsung.xlsx",sheetname="Teste")

def sempontuacao(text):  #função que retira todos os sinais(emocotions continuam)
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

data["Treinamento"] = data['Treinamento'].apply(sempontuacao)
datatest["Teste"] = datatest["Teste"].apply(sempontuacao)
data.Treinamento=data.Treinamento.str.replace("  "," ")#retirando espaços vazios
datatest["Teste"] = datatest.Teste.str.replace("  "," ")
data.Treinamento=data.Treinamento.str.replace("\n"," ")#retirando espaços vazios
datatest["Teste"] = datatest.Teste.str.replace("\n"," ")


lista_rel = []
lista_nrel = []
lista_total = []



for i in range(len(data["Treinamento"])):
    if data["Relevancia"][i]=="Relevante":
        lista_rel+=(list(set(data["Treinamento"][i].split())))
        lista_total+=(list(set(data["Treinamento"][i].split())))
    else:
        lista_nrel+=(list(set(data["Treinamento"][i].split())))
        lista_total+=(list(set(data["Treinamento"][i].split())))
        
        
#lista_rel=  list(set(lista_rel))
#lista_nrel = list(set(lista_nrel))

lista_total=list(set(lista_total))

n_rel = len(lista_rel)
n_nrel= len(lista_nrel)




        

        
    


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [143]:
def remover_outliers(lista):
    media=sum(lista)/len(lista)
    lista2=[]
    for i in range(len(lista)):
        if abs(lista[i]-media)<(10*media):
            lista2.append(lista[i])
    return lista2

palavra_prob_relevante=[]
final_relevante=[1]
palavra_prob_nrelevante=[]
final_nrelevante=[1]

for i in range(len(datatest.Teste)):
    palavra_prob_relevante.append([]) 
    palavra_prob_nrelevante.append([]) 
    for j in range(len(datatest.Teste[i].split())):
        count=lista_rel.count(datatest.Teste[i].split()[j])
        count2=lista_nrel.count(datatest.Teste[i].split()[j])
           
        palavra_prob_relevante[i].append((count+1)/(n_rel+len(lista_total)))
        palavra_prob_nrelevante[i].append((count2+1)/(n_nrel+len(lista_total)))      
        
        #print("PALAVRA:{0} FRASE:{1}".format(datatest.Teste[i].split()[j],i))
        #print(result_prob[i][j])
        
        final_relevante[i]=(final_relevante[i]*palavra_prob_relevante[i][j])
        final_nrelevante[i]=(final_nrelevante[i]*palavra_prob_nrelevante[i][j])
        
    final_relevante.append(1)
    final_nrelevante.append(1)       
        
        
falso_positivo=0
verdadeiro_positivo=0
falso_negativo=0
verdadeiro_negativo=0

muito_relevante=0 
relevante=0
neutro=0
irrelevante=0
muito_irrelevante=0


prob_rel=[]
prob_nrel=[]

for i in range(len(datatest["Relevancia"])):
   
    if final_relevante[i]>final_nrelevante[i]:
        
        prob_rel.append(final_relevante[i])
        if datatest.Relevancia[i]=="Relevante":
            
            verdadeiro_positivo+=1
        else:
            falso_positivo+=1        
            
            
    else:
        if datatest.Relevancia[i]=="Não Relevante":
            
            prob_nrel.append(final_nrelevante[i])
            verdadeiro_negativo+=1
            
        else:            
            falso_negativo+=1
               
prob_rel=remover_outliers(prob_rel)
print(prob_rel)
prob_nrel=remover_outliers(prob_nrel)


media_relevancia = sum(prob_rel)/len(prob_rel)
media_irrelevancia = sum(prob_rel)/len(prob_rel)

print(media_relevancia)

for i in range(len(prob_rel)):
    if final_relevante[i]>media_relevancia:
        muito_relevante+=1
    else:
        if final_relevante[i]>(0.1*media_relevancia):
            relevante+=1
        else:
            neutro+=1

for i in range(len(prob_nrel)):
    if final_nrelevante[i]>media_irrelevancia:
        muito_irrelevante+=1
    else:
        if final_nrelevante[i]>(0.1*media_irrelevancia):
            irrelevante+=1
        else:
            neutro+=1




p_falso_positivo=falso_positivo/datatest["Teste"].shape[0]
p_verdadeiro_positivo=verdadeiro_positivo/datatest["Teste"].shape[0]
p_falso_negativo=falso_negativo/datatest["Teste"].shape[0]
p_verdadeiro_negativo=verdadeiro_negativo/datatest["Teste"].shape[0]

print("Porcentagem de positivos verdadeiros :{0:.2f}%".format(p_verdadeiro_positivo*100))
print("Porcentagem de negativos verdadeiros: {0:.2f}%".format(p_verdadeiro_negativo*100))
print("Porcentagem de positivos falsos: {0:.2f}%".format(p_falso_positivo*100))
print("Porcentagem de negativos falsos : {0:.2f}%".format(p_falso_negativo*100))
#print()
#print()
#
#print("Porcentagem de muito relevantes :{0:.2f}".format((muito_relevante/len(datatest["Relevancia"]))*100))
#print("Porcentagem de relevantes: {0:.2f}".format((relevante/len(datatest["Relevancia"]))*100))
#print("Porcentagem de neutros: {0:.2f}%".format((neutro/len(datatest["Relevancia"]))*100))
#print("Porcentagem de irrelevantes : {0:.2f}%".format((irrelevante/len(datatest["Relevancia"]))*100))   
#print("Porcentagem de muito irrelevantes : {0:.2f}%".format((muito_irrelevante/len(datatest["Relevancia"]))*100))

print(muito_relevante)
print(relevante)
print(neutro)
print(irrelevante)
print(muito_irrelevante)




[8.49918133319608e-22, 1.5298526399752945e-20, 1.3796082531454214e-53, 1.064632612905422e-27, 8.150822739403942e-76, 2.1459199292087946e-28, 7.884805603334492e-50, 1.0044487030140821e-23, 2.932873489204252e-57, 1.1253692759517946e-82, 5.93117172960923e-39, 9.977251814051758e-62, 1.0810115761808905e-24, 2.3253760127624476e-18, 1.4400165080704554e-30, 1.6128011461366006e-56, 3.2518969000753134e-59, 4.099099698288363e-68, 1.1556872212887415e-48, 4.823484280590693e-99, 3.1489466839491476e-18, 1.093845113980668e-36, 1.5382196915353148e-39, 3.319144758749126e-51, 1.9301808573783873e-31, 9.911623043691633e-46, 3.0100752736544507e-48, 3.336455482039784e-27, 2.1768957038102116e-34, 2.575965990609379e-41, 5.578617518161701e-54, 3.4154019217242904e-60, 2.871343424199253e-36, 7.448005292922494e-52, 2.890139653876789e-48, 6.506088189977577e-26, 6.621622589542744e-72, 1.390436188113017e-47, 5.93117172960923e-39, 1.2773385077401881e-46, 3.5863620053377343e-75, 1.8307257299122474e-66, 1.37960825314542

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
